# Grab glassdoor links and review amounts

### Load dataframe

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.parse import unquote
import random
import time
from pathlib import Path
from torpy.http.requests import TorRequests

pd.options.mode.chained_assignment = None  # default='warn'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
input = pd.read_csv(r"W:\019_Glassdoor\1 Data\1 Glassdoor Links\0813_UniqueFirmList.csv", sep=";", decimal=",", encoding='unicode_escape')

## yahoo scraper

#### proxy rotation via tor

In [3]:
def yahoo_scraper(df, sleeptime_max=1, tor_active="no"):
    try: #if operation fails, then return the current df
        firmname = df["CompanyName"]
        print(f"Firm #{df.name + 1}: {firmname}")
        # query = f'site:glassdoor.com {firmname}'.replace(' ', '+')
        query = f'site:glassdoor.com "{firmname}"'.replace(' ', '+') #putting {firmname} around "" might help. however, the query might also be too strict not returning any results
        URL = f"https://search.yahoo.com/search?p={query}&vc=en&pz=20"

        ## shuffle through user agent list for bot prevention. use IP shuffling to prevent further bot detection
        time.sleep(round(random.uniform(0.25,sleeptime_max), 3))
        USER_AGENTS = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:76.0) Gecko/20100101 Firefox/76.0",
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:103.0) Gecko/20100101 Firefox/103.0",
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36",
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36 Edg/103.0.1264.71"
                    ]
        headers = {"user-agent":random.choice(USER_AGENTS)}

        ## use tor
        if tor_active=="yes":
            url = sess.get(URL, headers=headers)
        ## use regular IP
        else:
            url = requests.get(URL, headers=headers)

        soup = BeautifulSoup(url.content, "html.parser")
        time.sleep(0.2) #to fully load the page

        ## scraper ##
        #############

        link_found = 0
        ## check if amount of ratings is displayed in at least of the search results. if so, grab this number and the parent div containing the link
        if soup.find("li", {"class":["tc", "bxz-bb"]}) and len(soup.find("li", {"class":["tc", "bxz-bb"]}).text)<15: #check length of the classes text, since class is not exclusively used for amount of ratings
            ## grab the review number
            review_amount = soup.find("li", {"class":["tc", "bxz-bb"]}) #only get the first item, since it appears to be most accurate. could be of the last search result
            try: #sometimes "Currency: EUR" is displayed instead of the review amount
                review_amount_final = review_amount.text.split("(")[1][:-1] #"4/5 (3)"
                if "K" in review_amount_final: # "5.7K" to 5700
                    review_amount_final = review_amount_final.replace(".", "")
                    review_amount_final = review_amount_final.replace("K", "00")
                # print(f"review amount: {review_amount_final}")
                df["review_amount"] = review_amount_final
                ## grab the href from the parent of the same search result
            except:
                df["review_amount"] = "" 
            review_parents = review_amount.find_parents("div")
            for link in review_parents:
                if link.find("a") and "glassdoor" in link.find("a")["href"]:
                    alink = link.find("a")
                    link_found = 1
                    break
        ## if amount of ratings is not on page, then just grab the href
        elif link_found != 1:
            # print("review amount: none found")
            results = soup.findAll("h3")
            for link in results:
                if link.find("a") and "glassdoor" in link.find("a")["href"]:
                    alink = link.find("a")
                    link_found = 1
                    break
        ## if neither link nor review amount was found
            if link_found != 1:
                link_found = "no link found" #"no link found"
                link_final = ""
                print("no link found")

        
        ## decode href link if link was found ##

        if link_found == 1:
            link_with_yahoo = unquote(alink["href"]) #unquote decodes URLs
            try: #weird cases in which an image link is retrieved
                link_final = link_with_yahoo.split("RU=")[1].split("/RK=2")[0] #https://r.search.yahoo.com/_ylt=A0geK9dD9t9i2BIAtodXNyoA;_ylu=Y29sbwNiZjEEcG9zAzEEdnRpZAMEc2VjA3Ny/RV=2/RE=1658873541/RO=10/RU=https://www.glassdoor.com/Overview/Working-at-California-Micro-Devices-EI_IE1221.11,35.htm/RK=2/RS=Ss7CVR85Jhnt899NkWVSw9eSVUw-
                if "/Working" in link_final:
                    # from https://www.glassdoor.com/Overview/Working-at-Tesla-EI_IE43129.11,16.htm
                    # to https://www.glassdoor.com/Reviews/Tesla-Reviews-E43129.htm
                    link_final = link_final.replace("/Overview/Working-at-", "/Reviews/") #shouldnt be an issue to replace the /Overview/ part with /Reviews/ even though the original Reviews link looks different
                    link_final = link_final.replace("EI_IE", "Reviews-E") 
                    link_final = link_final.split(".htm")[0][:-6] + ".htm"
                    link_found = "transformed overview link"
                elif "/Jobs/" in link_final:
                    link_final = link_final.replace("/Jobs/", "/Reviews/")
                    link_final = link_final.replace("-Jobs-", "-Reviews-")
                    link_found = "transformed jobs link"
                elif "/Salary/" in link_final:
                    link_final = link_final.replace("/Salary/", "/Reviews/")
                    link_final = link_final.replace("-Salaries-", "-Reviews-")
                    link_found = "transformed salary link"
                elif "/Reviews/" in link_final:
                    link_found = "original review link"
                else:
                #elif "/job-listing/" or "/Job/" in link_final:
                    link_found = "wrong link"
            except:
                link_final = link_with_yahoo
                
        df["Link_Hannes"] = link_final
        df["link_found"] = link_found
        print(link_final, "\n")

        return df

    except:
        return df

# yahoo_scraper("El Paso Pipeline Partners")

In [5]:
## set parameters
startrow, endrow = 12000, 13000
tor = "yes" # change to yes or no
sleeptime_max = 0.5 #use >2 with original ip (tor = "no"). around 1000 requests possible until temp. ip block


## create a copy of the input frame; keep old index as extra column
input_copy = input[startrow:endrow].reset_index(drop=False)

input_copy["Link_Hannes"] = np.nan
input_copy["review_amount"] = np.nan
input_copy["link_found"] = np.nan

## execute yahoo scraper for each row by using apply ##
#######################################################

## with Tor-Gateway
if tor == "yes":
    with TorRequests() as tor_requests:
        print("Connecting to tor..")
        with tor_requests.get_session() as sess: #sess pulls a request of the page with the tor connection
            current_ip = sess.get("http://httpbin.org/ip").json()
            print(f"Current IP:{current_ip}; sleeptime: {sleeptime_max}\n")
            input_copy = input_copy.apply(yahoo_scraper, args=(sleeptime_max, "yes"), axis=1) #sess method can be executed from this sibling function
## without tor
else:
    input_copy = input_copy.apply(yahoo_scraper, args=(sleeptime_max, "no"), axis=1)
print(f"Scraping finished for {startrow} to {endrow}")
    

## reorder columns
# input_copy.keys() #['index', 'CompanyName', 'Ticker', 'ISIN', 'Link_Marius', 'Max_Cosine_Similarity', '2008_2022', 'EnglishSpeaking', 'Link_Hannes', 'review_amount', 'link_found']
input_copy = input_copy[['index', 'ISIN', 'Ticker', '2008_2022', 'EnglishSpeaking', 'CompanyName', 'Max_Cosine_Similarity', 'Link_Marius', 'Link_Hannes', 'link_found', 'review_amount']]


## save file with dynamic file name
path_with_time = Path.joinpath(Path.cwd().parent, f'{time.strftime("%m%d")}_UniqueFirmList_HannesLinks_{startrow}-{endrow}_strict.csv')
#"W:\019_Glassdoor\1 Data\1 Glassdoor Links\0808_UniqueFirmList_HannesLinks_1500-1509.csv"
input_copy.to_csv(path_with_time, sep=";", decimal=",", index=False)
print(f"{path_with_time} saved")

Connecting to tor..
Current IP:{'origin': '185.220.101.32'}; sleeptime: 0.5

Firm #1: Mid-America Apartment Communities 


c:\Users\g21728\AppData\Local\Programs\Python\Python37\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,


https://www.glassdoor.com/job-listing/apartment-maintenance-technician-maa-JV_IC1155583_KO0,32_KE33,36.htm?jl=1007168498537 

Current IP:{'origin': '185.220.101.32'}
Firm #2: ICC Labs
no link found
 

Current IP:{'origin': '185.220.101.32'}
Firm #3: DXC United Pty
no link found
 

Current IP:{'origin': '185.220.101.32'}
Firm #4: China Nutrifruit Group
no link found
 

Current IP:{'origin': '185.220.101.32'}
Firm #5: China Opportunity Acquisition
no link found
 

Current IP:{'origin': '185.220.101.32'}
Firm #6: MACOM Technology Solutions 
https://www.glassdoor.com/Reviews/MACOM-Reviews-E19802.htm 

Current IP:{'origin': '185.220.101.32'}
Firm #7: Cell Source 
https://www.glassdoor.com/job-listing/research-assistant-ii-baylor-college-of-medicine-JV_IC1140171_KO0,21_KE22,48.htm?jl=1008071962290 

Current IP:{'origin': '185.220.101.32'}
Firm #8: RTS Oil 


Stream #11: closed (but received b'i\xc9\xc3\x81\x99\x0bs\x15\xd8,\x9b\xe8\xb7Kk\xe2\xf1\x16A\x90\xac\xf3\xeb\xa7\x9e\xb8E\xc6\x10\x9b\xc1\xa2K\xd0-\xf8U\x96oo|\x0e]Y+\xd5\x92#\xac\xc2%\xb3x\x10\x18\xc6\x02s\xb4\xa1\xc5o\xbc\x1a\xf0\xbc\xbdF0\xb7\xcc"\x15\xb0\xd6\xd5\xcd\\.\xf9#\x9co\xeb!\x88\x96"Tb\xcfJ\xd4\x9aU\xca\x12\x01a+\xb0\x08\x13\x94D:9xF@\x06\x83[^W\x847\xf4D@\x13|P\xe3\xbc\\\x8b\xd3\x9a8L\xbf\x8e\x9c\x18\x80\x829\x98\xf9\xebNk\xcc\x12\t\xa4\x19v,=M\x90\xe0*\x95g\xcc\xef9\x17q\x08\xc3o!\xba\xbf\xbefQ\xd4\xcc4[G\x97\x9c\xd5\x9f\x1d\xe86h\xb4M\x95\xbf\xefY\xee\xdf\x82\xab\xd5L\xe1eT\x10\xc8\x18%\x9b@\xd9\xe2P\x96\x07\xf1u\x18aI\xac\x86\xdb\xa8G&`\x98\x1b^GN\xf1y\x8a\x15\x90\x08\x0e G%\x84x\x9e\x07\n\xeb\xb7\x07\xf5\xe5h\x89\xbc#E{\x07\xe6\xe7\xa3\x18~u\xcd\xd3:\xd0\xdb\xab\xd9P!T\xda\xf9o\x84\xb5\xec\xe12\xa7X\x179\xdfmJ\xc5\xe3\x95mw\xc2\xb6\x80\xac\xad\x94$P\xe2\xb0l\x91\x1aI\x96y$\x19\xb3\xf6\xb3\xbf\xca\xcc\xd3\xa52n\x82\xd8;\xe7\xd4\xc0\xc3~>\xd1\xc8\xdb\x08QKA\xe4F{,E\xf4

Firm #9: BioTransplant Incorporated


Stream #11: closed already
Stream #11 is already closed or was never opened (but received CellRelay(inner_cell = CellRelayData(data = ... (498 bytes)), stream_id = 11, digest = b'H\xcc<\x9b', circuit_id = 80000005))
Stream #11 is already closed or was never opened (but received CellRelay(inner_cell = CellRelayData(data = ... (498 bytes)), stream_id = 11, digest = b'\xec\xf3p\x06', circuit_id = 80000005))
Stream #11 is already closed or was never opened (but received CellRelay(inner_cell = CellRelayData(data = ... (498 bytes)), stream_id = 11, digest = b'-\xcd\xf6\xea', circuit_id = 80000005))
Stream #11 is already closed or was never opened (but received CellRelay(inner_cell = CellRelayData(data = ... (498 bytes)), stream_id = 11, digest = b'\xe4\xa8\x03\xb6', circuit_id = 80000005))
Stream #11 is already closed or was never opened (but received CellRelay(inner_cell = CellRelayData(data = ... (498 bytes)), stream_id = 11, digest = b'%:D\xbd', circuit_id = 80000005))
Stream #11 is alrea

Firm #10: Inca Minerals 
